# Image to Parquet
Convert images to binary and save them into a parquet file.

In [ ]:
import site
from pathlib import Path
from pyspark.sql import SparkSession

%load_ext autoreload
%autoreload 2

In [ ]:
def get_gcs_connector_jar() -> str:
    # Assuming the JAR is installed in the user site-packages of PySpark
    user_site_packages = site.getusersitepackages()
    jars_dir = Path(user_site_packages) / "pyspark" / "jars"
    # Search for the GCS connector JAR
    jar = [jar for jar in jars_dir.glob("gcs-connector-hadoop3-*.jar")]
    return str(jar[0])


gcs_connector_jar = get_gcs_connector_jar()
# Initialize Spark Session
spark = (
    SparkSession.builder.appName("Image2Parquet")
    .master("local[*]")
    .config("spark.jars", gcs_connector_jar)
    .getOrCreate()
)
print(spark)

In [ ]:
# Set Hadoop configurations for GCS
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set(
    "fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem"
)
sc._jsc.hadoopConfiguration().set(
    "fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
)

In [ ]:
import os
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# Base directory using pathlib
curr_dir = Path(os.getcwd())
base_dir = curr_dir.parents[1]
data_dir = base_dir / "data" / "SnakeCLEF2023-small_size" / "2023"
num_folders = 20

# Ensure base directory exists
if not data_dir.is_dir():
    raise FileNotFoundError(f"Data directory {data_dir} does not exist.")

# Getting subfolders
subfolders = sorted([f.name for f in data_dir.iterdir() if f.is_dir()])[:num_folders]

In [ ]:
# Schema for the DataFrame
from pyspark.sql.types import StructType, StructField, BinaryType, StringType

schema = StructType(
    [
        StructField("path", StringType(), True),
        StructField("folder_name", StringType(), True),
        StructField("year", StringType(), True),
        StructField("binomial_name", StringType(), True),
        StructField("file_name", StringType(), True),
        StructField("image_binary_data", BinaryType(), True),
    ]
)


# Function to convert image to binary
def image_to_binary(image_path):
    with open(image_path, "rb") as file:
        return file.read()

In [ ]:
# Create an empty RDD
image_rdd = spark.sparkContext.emptyRDD()

# Loop through subfolders and process images
for folder in subfolders:
    folder_path = data_dir / folder
    for img_name in os.listdir(folder_path):
        img_path = folder_path / img_name
        relative_path = img_path.relative_to(base_dir)  # Get relative path
        relative_path = str(relative_path).split("data/")[-1]
        folder_name = relative_path.split("/")[0]
        year = relative_path.split("/")[1]
        binomial_name = relative_path.split("/")[2]
        file_name = relative_path.split("/")[-1]
        # print(f"{folder_name}, {year}, {binomial_name}, {file_name}")
        binary_data = image_to_binary(str(img_path))
        image_rdd = image_rdd.union(
            spark.sparkContext.parallelize(
                [
                    (
                        relative_path,
                        folder_name,
                        year,
                        binomial_name,
                        file_name,
                        binary_data,
                    )
                ]
            )
        )

In [ ]:
# Convert RDD to DataFrame
image_df = spark.createDataFrame(image_rdd, schema)

# Show the first few rows of image_df
image_df.show(n=5)

In [ ]:
# Base directory for parquet_files folder
data_dir = Path(os.getcwd()).parents[1] / "data"

# Create "parquet_files" directory if it doesn't exist
parquet_dir = data_dir / "parquet_files"
os.makedirs(parquet_dir, exist_ok=True)

In [ ]:
# Path to the Parquet file
parquet_file_path = parquet_dir / "images_data.parquet"

# Save the DataFrame as a Parquet file
image_df.write.mode("overwrite").parquet(str(parquet_file_path))

In [ ]:
image_df.printSchema()

In [ ]:
def get_size_of_parquet(dir_path):
    total_size = 0
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            total_size += os.path.getsize(os.path.join(root, file))
    return total_size


# Get the size of the Parquet file (directory)
parquet_size = get_size_of_parquet(parquet_file_path)
print(f"Size of Parquet file: {parquet_size} bytes")

In [ ]:
# Define the GCS path
gcs_path = "gs://dsgt-clef-snakeclef-2024/data/parquet_files/image_data"

# Write the DataFrame to GCS
image_df.write.mode("overwrite").parquet(gcs_path)

## Load DataFrame from GCS

In [ ]:
from snakeclef.utils import get_spark

spark = get_spark()
display(spark)

In [ ]:
import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def display_images_from_binary(image_data_list, binomial_names, grid_size=(3, 3)):
    """
    Display images in a grid with binomial names as labels.

    :param image_data_list: List of binary image data.
    :param binomial_names: List of binomial names corresponding to each image.
    :param grid_size: Tuple (rows, cols) representing the grid size.
    """
    # Unpack the number of rows and columns for the grid
    rows, cols = grid_size

    # Create a matplotlib subplot with the specified grid size
    fig, axes = plt.subplots(rows, cols, figsize=(12, 12), dpi=80)

    # Flatten the axes array for easy iteration if it's 2D
    axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]

    for ax, binary_data, name in zip(axes, image_data_list, binomial_names):
        # Convert binary data to an image and display it
        image = Image.open(io.BytesIO(binary_data))
        ax.imshow(image)
        name = name.replace("_", " ")
        ax.set_xlabel(name)  # Set the binomial name as xlabel
        ax.xaxis.label.set_size(14)  # Set the font size for the xlabel
        ax.set_xticks([])
        ax.set_yticks([])
    plt.tight_layout()
    plt.show()

In [ ]:
# Collect binary image data from DataFrame
rows, cols = 3, 3
image_data_list = [row["image_binary_data"] for row in df.limit(rows * cols).collect()]
binomial_names = [row["binomial_name"] for row in df.limit(rows * cols).collect()]

# Display the images in a grid with binomial names
display_images_from_binary(image_data_list, binomial_names, grid_size=(3, 3))

## Join image DataFrame with Metadata files

In [ ]:
import os
from pathlib import Path
from pyspark.sql.functions import lit, regexp_extract, regexp_replace
from pyspark.sql import Row
from pyspark.sql import functions as F
from snakeclef.utils import get_spark

spark = get_spark()
display(spark)

In [ ]:
# Base directory using pathlib
curr_dir = Path(os.getcwd())
base_dir = curr_dir.parents[1]
base_dir = base_dir / "data" / "SnakeCLEF2023-small_size"

# Load all files from the base directory as binary data
# Convert Path object to string when passing to PySpark
image_df = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .option("recursiveFileLookup", "true")
    .load(str(base_dir))
)

# Construct the string to be replaced - adjust this based on your actual base path
to_remove = "file:" + str(base_dir.parents[0])

# Extract metadata from the file path
image_final_df = (
    image_df.withColumn("path", regexp_replace("path", to_remove, ""))
    .withColumn("folder_name", lit("SnakeCLEF2023-small_size"))
    .withColumn("year", regexp_extract("path", ".*/(\\d{4})/.*", 1))
    .withColumn("binomial_name", regexp_extract("path", ".*/(\\d{4})/(.*)/.*", 2))
    .withColumn("file_name", regexp_extract("path", ".*/([^/]+)$", 1))
)

# Select and rename columns to fit the target schema, including renaming 'content' to 'image_binary_data'
image_final_df = image_final_df.select(
    "path",
    "folder_name",
    "year",
    "binomial_name",
    "file_name",
    image_final_df["content"].alias("image_binary_data"),
)

# Create a new column "image_path" by removing "/SnakeCLEF2023-small_size/" from "path"
image_final_df = image_final_df.withColumn(
    "image_path", regexp_replace("path", "^/SnakeCLEF2023-small_size/", "")
)

# Print Schema
image_final_df.printSchema()

In [ ]:
image_final_df.show(n=3, truncate=100)

### join image_df with metadata from GCS

In [ ]:
# Get list of stored filed in cloud bucket
! gcloud storage ls gs://dsgt-clef-snakeclef-2024/raw

In [ ]:
# Root directory
raw_root = "gs://dsgt-clef-snakeclef-2024/raw/"

train_meta_hm = spark.read.csv(
    f"{raw_root}/SnakeCLEF2023-TrainMetadata-HM.csv", header=True, inferSchema=True
)
train_meta_inat = spark.read.csv(
    f"{raw_root}/SnakeCLEF2023-TrainMetadata-iNat.csv", header=True, inferSchema=True
)

train_meta_hm.printSchema()
train_meta_inat.printSchema()

In [ ]:
meta_df = (
    (
        # make this table consistent with the inat one
        train_meta_hm.withColumn(
            "observation_id", F.split("observation_id", " ")[1].cast("int")
        ).select(train_meta_inat.columns)
    )
    .union(train_meta_inat)
    .dropDuplicates()
    .repartition(1)
).cache()
meta_df.show()

In [ ]:
# check if there are duplicate ids
print(meta_df.count())
print(meta_df.groupBy("observation_id").count().where("count > 1").count())

# check if there are duplicate paths?
meta_df.groupBy("image_path").count().where("count > 1").show()

In [ ]:
# let's grab all the paths
train_root = Path("/mnt/data/SnakeCLEF2023-small_size")
paths = sorted([p.relative_to(train_root) for p in train_root.glob("**/*.jpg")])
len(paths), paths[:10]

In [ ]:
path_df = spark.createDataFrame([Row(path=p.as_posix()) for p in paths])
path_df.show(n=5, truncate=100)

In [ ]:
meta_df.select("image_path").show(n=3, truncate=False)


def remove_leading_parent(col):
    """remove the leading parent directory from the path

    e.g. 1992/Lampropeltis_annulata/70994554.jpg turns into Lampropeltis_annulata/70994554.jpg
    """
    return F.regexp_replace(col, "^(.+?)\/", "")


# let's join the two tables
joined_meta_df = meta_df.withColumn("path", remove_leading_parent("image_path")).join(
    path_df.withColumn("path", remove_leading_parent("path")), on="path", how="right"
)

# Renaming columns in joined_meta_df before the join to avoid confusion
joined_meta_df = joined_meta_df.withColumnRenamed(
    "path", "meta_path"
).withColumnRenamed("binomial_name", "meta_binomial_name")

joined_meta_df.show(n=5)

In [ ]:
# any missing rows on the left will be filled with nulls
joined_meta_df.where("image_path is not null").count(), path_df.count()

In [ ]:
# Perform an inner join on the 'image_path' column
joined_df = image_final_df.join(joined_meta_df, "image_path", "inner")

# Now, if you wish to drop the renamed columns from joined_meta_df:
final_df = joined_df.drop("meta_path", "meta_binomial_name")

# Show the result to verify the join
final_df.show(n=3, truncate=100)

In [ ]:
final_df.count()

In [ ]:
final_df.printSchema()

### using images_to_parquet script

In [1]:
from snakeclef.images_to_parquet import main

image_to_parquet_df = main()
image_to_parquet_df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/05 01:10:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/05 01:10:55 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


root
 |-- image_path: string (nullable = true)
 |-- path: string (nullable = true)
 |-- folder_name: string (nullable = false)
 |-- year: string (nullable = true)
 |-- binomial_name: string (nullable = true)
 |-- file_name: string (nullable = true)
 |-- image_binary_data: binary (nullable = true)
 |-- observation_id: integer (nullable = true)
 |-- endemic: boolean (nullable = true)
 |-- code: string (nullable = true)
 |-- class_id: integer (nullable = true)
 |-- subset: string (nullable = true)



In [2]:
image_to_parquet_df.show(n=3, truncate=100)

+-------------------------------------------+---------------------------------------------------------------------+------------------------+----+------------------------+-------------+----------------------------------------------------------------------------------------------------+--------------+-------+----+--------+------+
|                                 image_path|                                                                 path|             folder_name|year|           binomial_name|    file_name|                                                                                   image_binary_data|observation_id|endemic|code|class_id|subset|
+-------------------------------------------+---------------------------------------------------------------------+------------------------+----+------------------------+-------------+----------------------------------------------------------------------------------------------------+--------------+-------+----+--------+------+
|2017/Dend

In [3]:
image_to_parquet_df.count()

68495